<a href="https://colab.research.google.com/github/KevinHern/AI-Crash-Course/blob/main/AI_Crash_Course_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks

[Presentation: AI Crash Course 05](https://view.genial.ly/619e85e4ae59f30d5cb303d0/presentation-ai-crashcourse05)

## 0) Preparations

In [ ]:
# ----- Libraries ----- #

# For graph plotting
import matplotlib.pyplot as plt
from tensorflow.math import confusion_matrix

# For dataset manipulation
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# For visualizing more complex graphs
import seaborn as sns

# Neural networks
import tensorflow as tf

# Global constant for training acceleration
AUTOTUNE = tf.data.AUTOTUNE

# Classification

## 1) Dataset Preparations

In [ ]:
# Loading Dataset and have a glimpse about it
raw_dataset = pd.read_csv("diabetes.csv")

# Brief Statistical Summary of the dataset
raw_dataset.describe()

In [ ]:
# Lets check columns
raw_dataset.columns

In [ ]:
# Summary of the dataset
raw_dataset.head()

In [ ]:
# Returns a form of (# rows, # columns)
raw_dataset.shape 

In [ ]:
# Lets make a copy
new_dataset = raw_dataset.copy()

# Lets check for null values
# df.dropna()
print(new_dataset.isna().sum())

# Dropping null rows
new_dataset = new_dataset.dropna()

# Checking new dataset
new_dataset.head()

In [ ]:
# Lets visualize the data
sns.pairplot(new_dataset[["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]], diag_kind="kde")

In [ ]:
# Splitting dataset into training and testing
train, test = train_test_split(new_dataset, test_size=0.2)

# Sepparating both sets into dependent and independent variables
independent_variables = list(raw_dataset.columns)
independent_variables.remove('Outcome')
dependent_variables = ['Outcome']

train_set = train[independent_variables]
train_target = train[dependent_variables]

test_set = test[independent_variables]
test_target = test[dependent_variables]

## 2) Callbacks Introduction

In [ ]:
'''
  CALLBACKS:
  These are functions that are called once a certain amount of epochs end. These come very handful to deal with overfitting
'''

# Lets load an extension that helps us to visualize the performance of our model
%load_ext tensorboard

# Lets create the callback for that
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs/model/', histogram_freq=1)

# Now lets create a callback that save our model every epoch
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('logs/model/', monitor='accuracy', verbose=1, save_best_only=True, mode='max',save_freq='epoch')

## 3) Underfit NN

In [ ]:
# Lets build the model. NOTE: this is the construction of the architecture of the model!
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(len(independent_variables))),
  tf.keras.layers.Dense(units=2, activation='relu'),
  tf.keras.layers.Dense(units=len(dependent_variables), activation='sigmoid')
  ])

# Now lets compile the model. NOTE: These are the finishing touches before having a fully functional model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

# Now lets train the model!
model.fit(train_set,
          train_target,
          epochs=5,
          batch_size = 128 ,
          validation_split=0.2,
          callbacks=[tensorboard_callback, checkpoint_callback]
        )

# Lets evaluate our model
model.evaluate(x=test_set, y=test_target, batch_size=128)

In [ ]:
# Lets visualize
%tensorboard --logdir "logs/model/" --port=8008

## 4) Overfit NN

In [ ]:
# Lets build the model. NOTE: this is the construction of the architecture of the model!
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(len(independent_variables))),
  tf.keras.layers.Dense(units=64, activation='relu'),
  tf.keras.layers.Dense(units=64, activation='relu'),
  tf.keras.layers.Dense(units=64, activation='relu'),
  tf.keras.layers.Dense(units=64, activation='relu'),
  tf.keras.layers.Dense(units=64, activation='relu'),
  tf.keras.layers.Dense(units=len(dependent_variables), activation='sigmoid')
  ])

# Now lets compile the model. NOTE: These are the finishing touches before having a fully functional model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

# Now lets train the model!
model.fit(train_set,
          train_target,
          epochs=50,
          batch_size = 128 ,
          validation_split=0.2,
          callbacks=[tensorboard_callback, checkpoint_callback]
        )

# Lets evaluate our model
model.evaluate(x=test_set, y=test_target, batch_size=128)

In [ ]:
# Lets visualize
%tensorboard --logdir "logs/model/" --port=8008

## 5) More balanced and accepted NN

In [ ]:
'''
  Lets introduce a new callback to counter overfitting: EarlyStopping
  This callback stop the training once a criteria has met. This prevents overfitting.
  Important parameters:
  1. monitor: the metric which the callback does monitor. Validation loss is the accepted metric to observer
  2. Min Delta: This is the threshold that the callback uses to tell if the model has improved or not.
  This is the difference that the metric should satisfy between epochs.
  3. Patience: This is how many epochs does the callback has to see the model has not improved and immediately stops
  training afterwards
'''
earlystopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5,)

In [ ]:
# Lets build the model. NOTE: this is the construction of the architecture of the model!
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(len(independent_variables))),
  tf.keras.layers.Dense(units=32, activation='relu'),
  tf.keras.layers.Dense(units=32, activation='relu'),
  tf.keras.layers.Dense(units=32, activation='relu'),
  tf.keras.layers.Dense(units=len(dependent_variables), activation='sigmoid')
  ])

# Now lets compile the model. NOTE: These are the finishing touches before having a fully functional model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

# Now lets train the model!
model.fit(train_set,
          train_target,
          epochs=100,
          batch_size = 128 ,
          validation_split=0.2,
          callbacks=[tensorboard_callback, checkpoint_callback, earlystopping_callback]
        )

# Lets evaluate our model
model.evaluate(x=test_set, y=test_target, batch_size=128)

In [ ]:
# Lets visualize
%tensorboard --logdir "logs/model/" --port=8008